In [1]:
import boto3
from boto.s3.connection import S3Connection
import boto
from boto.s3.key import Key
import sys
import os
import json
import pandas as pd
from collections import defaultdict
# import pyplot as plt
import ast 
%matplotlib inline
sys.path.insert(0,os.path.abspath('..'))
from utils.credentials import access_key, secret_access_key

In [2]:
client = boto3.client('s3')

conn = boto.s3.connect_to_region('us-east-2',
       aws_access_key_id=access_key,
       aws_secret_access_key=secret_access_key,
       is_secure=True,
       calling_format = boto.s3.connection.OrdinaryCallingFormat(),
       )

bucket = conn.get_bucket('persproj')

In [ ]:
[x.name for x in bucket.list() if '.json' in x]

In [4]:
import boto3
import json

s3 = boto3.resource('s3', aws_access_key_id=access_key,
       aws_secret_access_key=secret_access_key)

def read_json_from_s3(s3_bucket_name, file_name):
    content_object = s3.Object(s3_bucket_name, file_name)
    file_content = content_object.get()['Body'].read().decode('utf-8')
    data = [json.loads(str(item)) for item in file_content.strip().split('\n')]
    df = pd.DataFrame(data)
    return df

#### Read as CSV

In [10]:
%%time
user_df = pd.read_csv('s3://persproj/data/user_df.csv')
print('user_df finished')

# user_df = pd.read_csv('s3://persproj/data/user_df.csv')
# print('user_df finished')

# business_df = pd.read_csv('s3://persproj/data/business_df.csv')
# print('business_df finished')
# checkin_df = pd.read_csv('s3://persproj/data/checkin_df.csv')
# print('checkin_df finished')

# tip_df = pd.read_csv('s3://persproj/data/tip_df.csv')
# print('tip_df finished')
# user_df = pd.read_csv('s3://persproj/data/user_df.csv')
# print('user_df finished')

# photo_df = pd.read_csv('s3://persproj/data/photo_df.csv')
# print('photo_df finished')

SyntaxError: invalid syntax (<ipython-input-10-7f16ab2d941a>, line 3)

#### Read as JSON

In [ ]:
review_df

In [ ]:
1+1

In [3]:
%%time

review_df = read_json_from_s3('persproj','review.json')
print('business_df finished' )
# business_df = read_json_from_s3('persproj','business.json')
# print('business_df finished' )

'''

FIRST TIME USE ONLY

photo_df = read_json_from_s3('persproj','photo.json')
print('photo_df finished' )

business_df = read_json_from_s3('persproj','business.json')
print('business_df finished' )
checkin_df = read_json_from_s3('persproj','checkin.json')
print('checkin_df finished' )
photo_df = read_json_from_s3('persproj','photo.json')
print('photo_df finished' )
tip_df = read_json_from_s3('persproj','tip.json')
print('tip_df finished' )
user_df = read_json_from_s3('persproj','user.json')
print('user_df finished' )
'''

NameError: name 'read_json_from_s3' is not defined

In [5]:
len(review_df)

6685900

In [ ]:
# check that all dfs are there
dfs = [business_df, checkin_df, photo_df, tip_df, user_df]
for df in dfs:
    print(len(df))

#### Add everything to s3 bcket for easy access

In [6]:
%%time
review_df.to_csv('s3://persproj/data/review_df.csv', index=False)
'''

FIRST TIME USE ONLY

business_df.to_csv('s3://persproj/data/business_df.csv', index=False)
print('business_df finished')
checkin_df.to_csv('s3://persproj/data/checkin_df.csv', index=False)
print('checkin_df finished')
photo_df.to_csv('s3://persproj/data/photo_df.csv', index=False)
print('photo_df finished')
tip_df.to_csv('s3://persproj/data/tip_df.csv', index=False)
print('tip_df finished')
user_df.to_csv('s3://persproj/data/user_df.csv', index=False)
print('user_df finished')

'''

CPU times: user 3min 20s, sys: 1min 43s, total: 5min 4s
Wall time: 28min 47s


## Business dataset

- What kind of categories are most common ?

In [ ]:
business_df

In [ ]:
business_df['categories_list'] = business_df['categories'].apply(lambda x: x.split(',') if not pd.isnull(x) else None,1)

In [ ]:
def count_categories(series, series_type='list'):
    count_dict = defaultdict(int)
    if series_type == 'list':
        for row in series:
            if row is not None or not pd.isnull(row):
                for item in row:
                    count_dict[item.strip()] += 1
    else:
        for row in series:
#             print(row, row is not None or pd.notna(row) or pd.notnull(row))
            if not pd.isnull(row) or not pd.isna(row):
                row = ast.literal_eval(row)
                for item in row.items():
                    count_dict[item[0].strip()] += 1
    res_df = pd.DataFrame.from_dict(count_dict, orient='index')
    res_df.rename(columns={0: 'count'}, inplace=1)
    return res_df.sort_values('count', ascending=False)

In [ ]:
count_df = count_categories(business_df['categories_list'].values)

In [ ]:
count_df.head(50).plot.barh(y='count', figsize=(10,13))
# plt.xticks(rotation=60)

In [ ]:
# not all restaurants contain the word "Restaurant"
# we want to include all food related businesses

food_df = (business_df[
    (business_df['categories'].str.contains('Food',na=False)) |
     (business_df['categories'].str.contains('Restaurant',na=False))|
     (business_df['categories'].str.contains('Bar',na=False))
])

food_count_df = count_categories(food_df['categories_list'])

In [ ]:
not_food_df = (business_df[
    (~business_df['categories'].str.contains('Food',na=False)) &
     (~business_df['categories'].str.contains('Restaurant',na=False))&
     (~business_df['categories'].str.contains('Bar',na=False))
])
not_food_count_df = count_categories(not_food_df['categories_list'])

In [ ]:
# check the non food categories to make sure there is no food related categories msising

not_food_count_df[75:]

In [ ]:
len(food_df)

In [ ]:
food_df['stars'].hist()

In [ ]:
food_df.head()

In [ ]:
food_df.groupby('state').count().plot.bar(y='address', figsize=(15,6))

In [ ]:
# df.boxplot(boxprops= dict(linewidth=1.0, color='black')
# , whiskerprops=dict(linestyle='-',linewidth=1.0, color='black'))

food_df.boxplot(column='stars', by='state', figsize=(17,8),
                boxprops=dict(linewidth=2.0, color='black'),
                whiskerprops=dict(linestyle='-',linewidth=2.0, color='black'),
                grid=False
               )
# (by='state',)

Most of the star distributions are very similar across states that have substantial number of observations.

### Available Attributes

In [ ]:
count_att_df = count_categories(business_df['attributes'].values, 'dict')

In [ ]:
business_df.head()

## Tips dataset

In [ ]:
len(tip_df)

In [ ]:
len(tip_df['business_id'].unique())

### Tips new columns

In [ ]:
tip_df['text_len'] = tip_df['text'].apply(lambda x: len(x) if not pd.isnull(x) else 0 ,1)
tip_df['business_id_clean'] = tip_df['business_id'].apply(lambda x: x.strip())

In [ ]:
# text length histogram
tip_df['text_len'].hist(figsize=(10,6))

In [ ]:
business_df[business_df['business_id'] =='--9e1ONYQuAa-CB_Rrw7Tw']

In [ ]:
tip_df[tip_df['business_id'] == '--9e1ONYQuAa-CB_Rrw7Tw']

In [ ]:
print(len(set(business_df['business_id'])))
print(len(set(tip_df['business_id']))) # tip dataset does not contain all business ids from busines dataset
print(len(set(business_df['business_id']) & set(tip_df['business_id'].values)))

In [ ]:
# see how many reviews they have vs. tips 

(business_df.merge(
    tip_df.groupby('business_id', as_index=False).agg({'compliment_count': len}),
    left_on='business_id',
    right_on='business_id',
    how='inner'
))[['business_id', 'review_count', 'compliment_count']]

In [ ]:
print(tip_df['date'].max())
print(tip_df['date'].min())

## User dataset

In [ ]:
user_df.loc[0].T

In [ ]:
user_df['average_stars'].hist(figsize=(10,6))

In [ ]:
user_df['review_count'].hist(figsize=(10,6))

## Photo Dataset

In [ ]:
photo_df.head()

In [ ]:
photo_df['label'].unique()

- Another reason for restaurant closure is high rent charges. Adding rent pricing per region could help explain more restaurant closures.
- A change in population demographics in certain areas of a city can increase or decrease traffic to some restaurants.

### Sources for yelp dataset problems:

https://towardsdatascience.com/using-yelp-data-to-predict-restaurant-closure-8aafa4f72ad6